In [21]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pandas as pd
import pickle

# Define the Model Class --> same as used in training loop

class Model(nn.Module):
    def __init__(self , num_features):
        super(Model, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(num_features, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
    def forward(self, features):
        return self.network(features)


In [22]:
# Load the Saved Model
model = Model(num_features = 12)  
model.load_state_dict(torch.load("model.pth"))


<All keys matched successfully>

In [23]:

# Load the encoders and scaler
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [24]:
# Example input data
input_data = {
    'CreditScore': 800,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 3,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 500000
}

In [25]:
# One-hot encode 'Geography'
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()


c:\Users\hp\OneDrive\Desktop\annclassification\ccvenv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [26]:
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [27]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,800,France,Male,40,3,60000,3,1,1,500000


In [28]:
## Encode categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,800,France,1,40,3,60000,3,1,1,500000


In [29]:
input_df = pd.concat([input_df.drop('Geography' , axis = 1) , geo_encoded_df], axis=1)

In [30]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,800,1,40,3,60000,3,1,1,500000,1.0,0.0,0.0


In [31]:
input_scaled = scaler.transform(input_df)

In [32]:
input_scaled

array([[ 1.53956105,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         2.53355998,  0.64920267,  0.97481699,  6.94716943,  1.00150113,
        -0.57946723, -0.57638802]])

In [33]:
input_tensor = torch.tensor(input_scaled, dtype=torch.float32)

In [34]:
with torch.no_grad():
    output = model(input_tensor)
    prediction = (output > 0.5).float().item()
    print(f"Prediction: {prediction}")

Prediction: 0.0
